Implement a parallel BDDC preconditioner
====

Implement a MPI - parallel preconditioner

$$
C^{-1} = R \widetilde A^{-1} R^T.
$$

where $R$ is an averaging operator across sub-domains, and $\widetilde A$ consists of local inverses, and a coarse grid system.

* Team A:<br>
Implement the $R$ operator. Use bilinear-form $A(u,v) = \int \nabla u \nabla v + \frac{1}{H^2} u v$, and use local inverses for $\widetilde A$.

* Team B:<br>
Use bilinear-form $A(u,v) = \int \nabla u \nabla v$, and define and implement the $\widetilde A^{-1}$. Add mean-values for every sub-domain interface $\gamma_{ij}$ (one constant $u_{ij}$ per interface). Define the bilinear-form
$$
\tilde A(u,v) = \sum_{\Omega_i} \int_{\Omega_i} \nabla u \nabla v + \int_{\partial \Omega_i} \frac{1}{H^2} (u_i - u_{ij}) (v_i - v_{ij})
$$

In [ ]:
from ipyparallel import Client
c = Client()
c.ids

In [ ]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD

from ngsolve import *

from netgen.geom2d import unit_square
comm = MPI.COMM_WORLD
if comm.rank == 0:
    mesh = Mesh(unit_square.GenerateMesh(maxh=0.05).Distribute(comm))
else:
    mesh = Mesh(netgen.meshing.Mesh.Receive(comm))

In [ ]:
%%px
fes = H1(mesh, dirichlet='.*')
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx + 0.001*u*v*dx).Assemble()
f = LinearForm(10*x*v*dx).Assemble()
gfu = GridFunction(fes)

Building the R operator:

In [ ]:
%%px

pardofs = fes.ParallelDofs()
scaling = CreateVVector(pardofs.ndoflocal)
for i in range(pardofs.ndoflocal):
    scaling[i] = 1.0 / (1+len(pardofs.Dof2Proc(i)))
# print (scaling)

scalingmat = DiagonalMatrix(scaling)

# input: local vector, output: consistent global vector
def Averaging(x, y):
    y.local_vec.data = scalingmat * x
    y.SetParallelStatus(PARALLEL_STATUS.DISTRIBUTED)
    y.Cumulate()
 
# input: distributed vector, output: local vector
def AveragingT(x,y):
    x.Cumulate()   
    y.data = scalingmat * x.local_vec

Testing the averaging:

In [ ]:
%%px
hv = gfu.vec.local_vec.CreateVector()
hv.data = a.mat.local_mat.Inverse() * f.vec.local_vec
Averaging (hv, gfu.vec)

In [ ]:
%%px
class ParallelPreconditioner(BaseMatrix):
    def __init__ (self, atilde):
        super().__init__()
        self.atilde = atilde
        self.inv = atilde.local_mat.Inverse()
        
    def Mult (self, x, y):
        hv1 = x.local_vec.CreateVector()
        hv2 = x.local_vec.CreateVector()
        
        AveragingT(x,hv1)
        hv2.data = self.inv * hv1
        Averaging(hv2, y)

    def Shape (self):
        return self.atilde.shape
    def CreateVector (self, col):
        return self.atilde.CreateVector(col)

In [ ]:
%%px
pre = ParallelPreconditioner(a.mat)

from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre, printrates=comm.rank==0)
gfu.vec.data = inv * f.vec

from ngsolve.la import EigenValues_Preconditioner
lam = EigenValues_Preconditioner(a.mat, pre)
if comm.rank == 0:
    print ("lam min/max = ", min(lam), max(lam))

In [ ]:
from ngsolve.webgui import Draw

gfu = c[:]['gfu']
Draw(gfu[0]);

In [ ]:
%%px
from ngsolve.la import ParallelDofs
from ngsolve.la import SparseMatrixd

pardofs = fes.ParallelDofs()
exprocs = pardofs.ExchangeProcs()

skel_dof2proc = [ [p] for p in exprocs]
skel_pardofs = ParallelDofs(skel_dof2proc, comm)

hmat = a.mat.local_mat.CreateMatrix()
hmat.AsVector().data = a.mat.local_mat.AsVector()

coupling = MultiVector(hmat.CreateVector(True), len(exprocs))
couplingmat = BaseMatrix(coupling) # high rectangular
for vec in coupling:
    vec[:] = 0

dense = Matrix(len(exprocs), len(exprocs))
dense[:,:] = 0

H = 1
for nr, exproc in enumerate(exprocs):
    for dof in pardofs.Proc2Dof(exproc):
        hmat[dof,dof] += 1/H
        coupling[nr][dof] += -1/H
        dense[nr,nr] += 1/H
        
        
hinv = hmat.Inverse()
Schur = InnerProduct(coupling, hinv*coupling)
dense -= Schur

if comm.rank != 0:
    coo = [(i,j,dense[i,j]) for i in range(dense.h) for j in range(dense.w)]
    i,j,val = zip(*coo)
else:
    i,j,val = [], [], []
    
sparseschur = SparseMatrixd.CreateFromCOO(indi=list(i),indj=list(j),values=list(val), h=dense.h, w=dense.w)
globschur = ParallelMatrix(sparseschur, skel_pardofs) 
globinv = globschur.Inverse()

# dummypardofs = ParallelDofs([[]]*fes.ndof, comm)
# globcoupling = ParallelMatrix(couplingmat, row_pardofs=skel_pardofs,
#                              col_pardofs=dummypardofs)
# ainv = hinv + globcouplingmat @ globinv @ globcouplingmat.T


In [ ]:
%%px
class ParallelPreconditioner2(BaseMatrix):
    def __init__ (self, invloc, invglob, coupling):
        super().__init__()
        self.invloc = invloc
        self.invglob = invglob
        self.coupling = coupling
        
    def Mult (self, x, y):
        hv1 = x.local_vec.CreateVector()
        hv2 = x.local_vec.CreateVector()
        hv1glob = self.invglob.CreateRowVector()
        hv2glob = self.invglob.CreateColVector()
        
        AveragingT(x,hv1)
        hv2.data = self.invloc * hv1
        hv1glob.local_vec.data = self.coupling.T * hv1
        hv2glob.data = self.invglob * hv1glob
        hv2.data += self.coupling * hv2.local_vec
        Averaging(hv2, y)

    def Shape (self):
        return self.atilde.shape
    def CreateVector (self, col):
        return self.atilde.CreateVector(col)

In [ ]:
%%px
pre = ParallelPreconditioner2(hinv, globinv, couplingmat)

from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre, printrates=comm.rank==0)
gfu.vec.data = inv * f.vec

from ngsolve.la import EigenValues_Preconditioner
lam = EigenValues_Preconditioner(a.mat, pre)
if comm.rank == 0:
    print ("lam min/max = ", min(lam), max(lam))

In [ ]:
from ngsolve import *
from ngsolve.la import *
dir()